In [1]:
# FB and Google


import pandas as pd
import numpy as np
from facebookads.adobjects.customaudience import CustomAudience
from facebookads.adobjects.adaccount import AdAccount
from facebookads.api import FacebookAdsApi
import datetime
import os
import glob



BigLotsAccount_id="act_271491453638620"

jian_APP_ID="2537704939796694"# jian's fb marketing app id
jian_token="EAAkEB3l7ONYBALOwTjZA6ZA69KGshjjZCghKd2FhqZCVvcZCxcQZCglQshts2EZCZBccbvwUddxKlje4ZCz3w8qkLWICYdS8cZCAw2e8VZCs9VHEt7sg5gR6gc4o8TVV73OAABUf7zKqoXLyeREavx7Viz8ArwVQxsDfMVSJK7o5qOqiQZDZD"

def creatfbaudience(file_path,ACCESS_TOKEN,accountid,description,audience_name):
    '''
    input:
    file_path: str, full_path, basename as the audience name on FB
        make sure the columns contains the "email_address_hash", use this col only
    
    ACCESS_TOKEN: personal token #app id not needed
    accountid: account_id starting with "act_", followed by the id str
    as Nov. 20, 2019, latest verision is 'v5.0'
    '''
    FacebookAdsApi.init(access_token=ACCESS_TOKEN, api_version='v5.0')
    myAccount = AdAccount(accountid)
    fields = []
    fbname=audience_name
    params = {
      'name': fbname,
      'subtype': 'CUSTOM',
      'description': description,
      'customer_file_source': 'USER_PROVIDED_ONLY',
    }
    fbname = myAccount.create_custom_audience(
      fields=fields,
      params=params,
    )['id']
    
    df=pd.read_csv(file_path,dtype=str,)
    listlen = df.shape[0]
    requestUrl = 'https://graph.facebook.com/v5.0/'+ fbname + '/users'
    
    chunck_size=50000
    i_counter=0
    for df_chunck in pd.read_csv(file_path,chunksize=chunck_size,dtype=str,usecols=["email_address_hash"]):
        payload = {}
        payload['schema'] = ['EMAIL']
        df_chunck=df_chunck.rename(columns={"email_address_hash":"EMAIL"})
        df_chunck=df_chunck.drop_duplicates()
        payload_data=df_chunck['EMAIL'].apply(lambda x: [x]).tolist()

        payload['data'] = payload_data
        params = {}
        params ={'payload':payload} 
        CustomAudience(fbname).create_user(
                      fields=fields,
                      params=params,
        )
        i_counter+=1
        print(i_counter,datetime.datetime.now())
  

    return 'All done: '+str(datetime.datetime.now())

In [2]:
list_missing_name=pd.ExcelFile("./piece_10_segs_missing_direct_to_FB/Big Lots Q1 Rewards Segmentation 1.29 Final - Not in FB .xlsx")
print(list_missing_name.sheet_names)

list_missing_name=list_missing_name.parse("Not in FB")
list_missing_name=list_missing_name['segment'].unique().tolist()
len(list_missing_name)

['Segment Summary', 'Pivot', 'df_0_to_18_count_all', 'Not in FB', 'df_19_to_48_count_all']


10

In [3]:
import glob

list_all_files=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/*.csv")
list_10_files=[]

for name_patter in list_missing_name:
    for file in list_all_files:
        if name_patter in os.path.basename(file):
            list_10_files.append(file)
print(list_10_files)
            

['/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Lapsed1318_L_P_Below50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv', '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Lapsed1318_L_P_Below50K_SingleFamily_CompetitorsHigh_FurinitureLow_RFM20Q1.csv', '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Lapsed1318_L_P_Below50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1.csv', '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Lapsed1318_L_P_Below50K_SingleFamily_CompetitorsLow_FurinitureLow_RFM20Q1.csv', '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments_uploaded_LR_0_18/T_SOTF_Lapsed1318_L_S_Below50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv', '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/final_segments

In [7]:
for local_file in list_10_files:
    creatfbaudience(file_path=local_file,
                    ACCESS_TOKEN=jian_token,
                    accountid=BigLotsAccount_id,
                    description="add the missing 10 files: "+os.path.basename(local_file).split(".")[0],
                    audience_name=os.path.basename(local_file).split(".")[0]
                   )

1 2020-01-31 11:20:52.463756
2 2020-01-31 11:20:53.228903
1 2020-01-31 11:20:54.684253
1 2020-01-31 11:20:57.777137
1 2020-01-31 11:20:59.065204
1 2020-01-31 11:21:02.877552
1 2020-01-31 11:21:04.948895
1 2020-01-31 11:21:06.604030
1 2020-01-31 11:21:07.948356
1 2020-01-31 11:21:09.508947
1 2020-01-31 11:21:11.189892


In [8]:
[os.path.basename(x) for x in list_10_files]

['T_SOTF_Lapsed1318_L_P_Below50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv',
 'T_SOTF_Lapsed1318_L_P_Below50K_SingleFamily_CompetitorsHigh_FurinitureLow_RFM20Q1.csv',
 'T_SOTF_Lapsed1318_L_P_Below50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1.csv',
 'T_SOTF_Lapsed1318_L_P_Below50K_SingleFamily_CompetitorsLow_FurinitureLow_RFM20Q1.csv',
 'T_SOTF_Lapsed1318_L_S_Below50K_SingleFamily_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv',
 'T_SOTF_Lapsed1318_L_S_Below50K_SingleFamily_CompetitorsLow_FurinitureHigh_RFM20Q1.csv',
 'T_SOTF_Lapsed1318_L_P_Below50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv',
 'T_SOTF_Lapsed1318_L_P_Below50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureLow_RFM20Q1.csv',
 'T_SOTF_Lapsed1318_L_P_Below50K_MultiFamilyOrCondo_CompetitorsLow_FurinitureHigh_RFM20Q1.csv',
 'T_SOTF_Lapsed1318_L_S_Below50K_MultiFamilyOrCondo_CompetitorsHigh_FurinitureHigh_RFM20Q1.csv']